In [1]:
from nipype.interfaces import niftyreg
import numpy as np
from skimage import io
from pathlib import Path
import re
import ants
from skimage.transform import resize
from tqdm import tqdm

In [18]:
directory_seg = Path('james_preds')
files_seg  = list(directory_seg.glob('*_seg.npy'))
files_seg = sorted([x.as_posix() for x in files_seg])

In [19]:
for i in tqdm(range(len(files_seg))):
    tmp = np.load(files_seg[i])
    tmp = np.reshape(tmp,(1,507,507,252))
    io.imsave(re.sub('.npy','.tif',files_seg[0]),tmp)

  0%|          | 0/94 [00:00<?, ?it/s]/tmp/ipykernel_237611/3179607572.py:4: UserWarning:

james_preds/XYZres006_0001_seg.tif is a low contrast image

100%|██████████| 94/94 [00:31<00:00,  2.94it/s]


In [20]:
files_seg  = list(directory_seg.glob('*_seg.tif'))
files_seg = sorted([x.as_posix() for x in files_seg])
files_seg[0]

'james_preds/XYZres006_0001_seg.tif'

In [2]:
image_path = Path('../TBI')
images = list(image_path.glob('vbm*/*?[0-9].tif'))
images = sorted([x.as_posix() for x in images])
len(images)

94

In [3]:
for i in tqdm(range(len(images))[::-1]):
    image = io.imread(images[i])
    image = np.swapaxes(image,0,1)
    image = np.swapaxes(image,1,3)
    image = resize(image,(2,507,507,252),preserve_range=True)
    zeros = np.zeros((1,507,507,252)).astype('uint16')
    image = np.append(image,zeros,axis = 0)
    image = image.astype('float16')
    io.imsave('james_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name),image)

 60%|█████▉    | 56/94 [33:32<22:45, 35.94s/it]


KeyboardInterrupt: 

In [50]:
images = list(directory_seg.glob('*_0001_upsamp.tif'))
images = sorted([x.as_posix() for x in images])
images[0]

'james_preds/XYZres006_0001_upsamp.tif'

In [40]:
%%time
fix = ants.image_read(re.sub('_0001','',images[0]))
mov = ants.image_read(images[0])
fix_mask = ants.image_read(re.sub('upsamp','seg',re.sub('_0001','',images[0])))
mov_mask = ants.image_read(re.sub('upsamp','seg',images[0]))
#mytx = ants.registratiom(fixed = fix,
#                        moving = mov,
#                        type_of_transform = 'rigid'
#                        )
#warpedmask = ants.apply_transforms(fixed = fix_mask
#                                   moving = mov_mask,
#                                   transformlist = mytx['fwdtransforms'],
#                                   interpolator = 'nearestNeighbor'
#                                  )

RuntimeError: /build103/ANTsPy/itksource/Modules/IO/TIFF/src/itkTIFFImageIO.cxx:503:
ITK ERROR: TIFFImageIO(0x1b9fa30): Unable to read tiff file: Sorry, can not handle images with 64-bit samples

In [52]:
np.max(io.imread(re.sub('_0001','',images[0])))

0.01560997940032044